## Data Prep


In [0]:
df = spark.table("wellness_data.mimilabs.landscape_special_needs_plan")

In [0]:
df.write.saveAsTable("wellness_data.mimilabs.starrating_measure_star")

In [0]:
df = spark.table("wellness_data.mimilabs.starrating_measure_star")
df.write.saveAsTable("wellness.starrating_measure_star")

## Fuctions to use

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.plans_by_state_and_type(
  state_code STRING COMMENT 'Two-letter state code',
  plan_type STRING COMMENT 'Plan type (e.g., Dual-Eligible)'
)
RETURNS STRING
COMMENT "Returns list of plan names and IDs in a state by plan type"
RETURN
  SELECT COLLECT_LIST(CONCAT(
    'Plan Name: ', plan_name, ', ',
    'Plan ID: ', plan_id
  ))
  FROM workspace.wellness.landscape_special_needs_plan
  WHERE state = state_code
    AND special_needs_plan_type = plan_type;


In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.top_rated_plans_by_state(
  state_code STRING COMMENT 'Two-letter state code'
)
RETURNS STRING
COMMENT "Returns top 5 plans by average star rating in the given state"
RETURN
  SELECT COLLECT_LIST(CONCAT(
    'Plan Name: ', p.plan_name, ', ',
    'Average Rating: ', CAST(AVG(s.measure_value) AS STRING)
  ))
  FROM workspace.wellness.landscape_special_needs_plan p
  JOIN wellness_data.mimilabs.starrating_measure_star s ON p.contract_id = s.contract_id
  WHERE p.state = state_code
  GROUP BY p.plan_name
  ORDER BY AVG(s.measure_value) DESC
  LIMIT 5;


In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.contracts_by_year(
  year INT COMMENT 'Performance year'
)
RETURNS STRING
COMMENT "Returns list of contracts and their organization types for a given performance year"
RETURN
  SELECT COLLECT_LIST(CONCAT(
    'Contract Name: ', contract_name, ', ',
    'Organization Type: ', organization_type, ', ',
    'Contract ID: ', contract_id
  ))
  FROM wellness_data.mimilabs.starrating_measure_star
  WHERE performance_year = year;


In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.lookup_measures_by_org(
  org_name STRING COMMENT 'Marketing name of the organization'
)
RETURNS STRING
COMMENT "Returns list of performance measures for the given organization"
RETURN
  SELECT COLLECT_LIST(CONCAT(
    'Measure Code: ', measure_code, ', ',
    'Measure Description: ', measure_desc, ', ',
    'Value: ', measure_value_raw, ', ',
    'Performance Year: ', CAST(performance_year AS STRING)
  ))
  FROM wellness_data.mimilabs.starrating_measure_star
  WHERE organization_marketing_name = org_name;


In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.health_plan_look_up(
  plan_type STRING COMMENT 'type of the plan whose plan types to look up'
)
RETURNS STRING
COMMENT "Returns list of plans about a plan typle given the type of plan "
RETURN SELECT COLLECT_LIST(CONCAT(
    'Plan_Name: ', plan_name, ', ',
    'Plan_ID: ', plan_id, ', ',
    'plan_type ', special_needs_plan_type, ', '
  ))
  FROM workspace.wellness.landscape_special_needs_plan
  WHERE special_needs_plan_type = plan_type;

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.health_plan_look_up_county(
  plan_name STRING COMMENT 'type of the plan whose plan to look up'
)
RETURNS STRING
COMMENT "Returns list of plans about a plan given the county of plan "
RETURN SELECT COLLECT_LIST(CONCAT(
    'Plan_Name: ', plan_name, ', ',
    'plan_country', county, ', '
  ))
  FROM workspace.wellness.landscape_special_needs_plan
  WHERE county = plan_name;

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.health_plan_look_up_starr(
  star_rating STRING COMMENT 'type of the rating whose plan to look up'
)
RETURNS STRING
COMMENT "Returns list of plans about a plan given the rating of plan "
RETURN SELECT COLLECT_LIST(CONCAT(
    'Plan_Name: ', plan_name, ', ',
    'plan_rating', overall_star_rating, ', '
  ))
  FROM workspace.wellness.landscape_special_needs_plan
  WHERE overall_star_rating = star_rating;

In [0]:
%sql
CREATE OR REPLACE FUNCTION workspace.wellness.health_plan_look_up_organization(
  special_needs_type STRING COMMENT 'type of the special_needs_type whose organization to look up'
)
RETURNS STRING
COMMENT "Returns list of organizations about a plan type given the type of plan "
RETURN SELECT COLLECT_LIST(CONCAT(
    'Organziation: ', organization_name, ', ',
    'plan_type', special_needs_plan_type, ', '
  ))
  FROM workspace.wellness.landscape_special_needs_plan
  WHERE organization_name = special_needs_type;